# Spark S3 Connection

we need to make sure the hadoop aws package is available when we load spark

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession


# depending on your set up:
# if you are running the spark app locally, set the driver memory to something your system can handle
# if you are running on a cluster, then also set the executor memory - if necessary (depends on how your cluster is configured)
conf = SparkConf()
conf.set('spark.executor.memory', '8g')
conf.set('spark.driver.memory', '8g')
conf.set('spark.jars', '/usr/local/spark/jars/aws-java-sdk-1.7.4.jar,/usr/local/spark/jars/hadoop-aws-2.7.4.jar')

spark_session = SparkSession.builder \
        .config(conf=conf) \
        .appName('s3-write') \
        .getOrCreate()
spark_session.sparkContext.setLogLevel("INFO")

### Reading config from file

We can use the configparser package to read the credentials from the standard aws file (https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html).

In [5]:
import configparser
import os
aws_profile = "lijo" #user profile name
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_key = config.get(aws_profile, "aws_access_key_id") 
secret_key = config.get(aws_profile, "aws_secret_access_key")
spark_session.sparkContext.getConf().getAll()

[('spark.driver.host', '192.168.0.49'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 's3-write'),
 ('spark.repl.local.jars',
  'file:///usr/local/spark/jars/aws-java-sdk-1.7.4.jar,file:///usr/local/spark/jars/hadoop-aws-2.7.4.jar'),
 ('spark.jars',
  '/usr/local/spark/jars/aws-java-sdk-1.7.4.jar,/usr/local/spark/jars/hadoop-aws-2.7.4.jar'),
 ('spark.app.id', 'local-1594186616260'),
 ('spark.rdd.compress', 'True'),
 ('spark.driver.memory', '8g'),
 ('spark.driver.port', '35497'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.memory', '8g'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true')]

### Configure hadoop

In [3]:
hadoop_conf=spark_session._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", access_key)
hadoop_conf.set("fs.s3a.secret.key", secret_key)
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.connection.maximum", "1000")
hadoop_conf.set("fs.s3.maxConnections", "1000")
hadoop_conf.set("fs.s3a.connection.establish.timeout", "50000")
hadoop_conf.set("fs.s3a.socket.recv.buffer", "8192000")
hadoop_conf.set("fs.s3a.readahead.range", "32M")

### Reading data from s3

In [4]:
# data = spark_session.sparkContext.textFile("s3a://pyspark-lijo-test/auction.csv").map(lambda line: line.split(","))
# print(data.count())
# for d in data.collect()[:3]:
#     print(d)
# from pyspark.sql.types import StructType
# from pyspark.sql.types import StructField
# from pyspark.sql.types import StringType

# schema = StructType([StructField(str(i), StringType(), True) for i in range(11)])
# print(schema)
# df = spark_session.createDataFrame(data, schema)
# df.show(2)

# df=sqlContext.read.csv("s3a://pyspark-lijo-test/vgsales.csv",header=True, inferSchema=True)
df = spark_session.read.csv(
    "s3a://pyspark-lijo-test/output/auction", header=True, inferSchema=True, mode="DROPMALFORMED")
# df.count()

# df = spark_session.read.format("s3selectCSV").load("s3a://pyspark-lijo-test/output/auction")
df.show(2)

Py4JJavaError: An error occurred while calling o43.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, 192.168.0.49, executor driver): org.apache.spark.util.TaskCompletionListenerException: Premature end of Content-Length delimited message body (expected: 888,879; received: 16,360)
	at org.apache.spark.TaskContextImpl.invokeListeners(TaskContextImpl.scala:145)
	at org.apache.spark.TaskContextImpl.markTaskCompleted(TaskContextImpl.scala:124)
	at org.apache.spark.scheduler.Task.run(Task.scala:143)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:444)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1377)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:447)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.r

In [15]:
# df.show(2)
# df.registerTempTable("auction")
# results = sql.sql(
#    """SELECT auctionid, MAX(price) as price, item FROM auction
#      GROUP BY item,auctionid"""
#    )

# results.show(10)
# results.write.csv("s3a://pyspark-lijo-test/output/test.csv",mode="overwrite")
# results.coalesce(10).write.mode("overwrite").format("csv").save("../output/test.csv")
df=spark_session.read.csv("../data/auction.csv",header=True, inferSchema=True)
df.show(2)
# df.write.csv("s3a://pyspark-lijo-test/output/auction",mode="overwrite")
df.coalesce(20).write.mode("overwrite").format("csv").save("s3a://pyspark-lijo-test/output/auction")
# df.write.save("s3a://pyspark-lijo-test/output", format='csv', header=True, mode="overwrite")


+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
| auctionid|  bid| bidtime|      bidder|bidderrate|openbid|price|              item| auction_type|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
|1638893549|175.0|2.230949|schadenfreud|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
|1638893549|100.0|2.600116|       chuik|         0|   99.0|177.5|Cartier wristwatch|3 day auction|
+----------+-----+--------+------------+----------+-------+-----+------------------+-------------+
only showing top 2 rows



In [1]:
df=sql.read.csv("s3a://pyspark-lijo-test/output/part-00000-d3398b25-38db-4e8e-ac76-9ff354a7917c-c000.csv",header=True, inferSchema=True)
df.show(2)

NameError: name 'sql' is not defined

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext('local[2]', 'my_app')
ssc = StreamingContext(sc, 1)

lines = ssc.textFileStream('../data/auction.csv')
counts = lines.map(lambda line: line.split(","))
counts.pprint()
ssc.start()
ssc.awaitTermination()
sc

-------------------------------------------
Time: 2020-07-08 10:42:50
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:51
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:52
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:53
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:54
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:55
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:56
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:57
-------------------------------------------

-------------------------------------------
Time: 2020-07-08 10:42:58
----------